In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.inks import InkGamut
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction
from chromalab.animation import Animation

objc[78587]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a67c2e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2c1e366f0). One of the two will be used. Which one is undefined.
objc[78587]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a67c2db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2c1e36718). One of the two will be used. Which one is undefined.
objc[78587]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2a67c2e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 10)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths, illuminant=d65) 
maxbasis = MaxBasis(trichromat, verbose=True)
viz = PSWrapper(trichromat, maxbasis, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

100%|██████████| 28/28 [00:00<00:00, 855.89it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 930/930 [00:02<00:00, 432.47it/s]


In [7]:
cmy_primaries_dict = {}
with open(f'../data/nix/PrintColors/000.csv') as csvf:
    spamreader = csv.reader(csvf, delimiter=';')
    for i, row in enumerate(spamreader): 
        if i == 4:
            color_data = np.array(row[33:],dtype=float)
            spectra = Spectra(data=color_data, wavelengths=wavelengths)
            cmy_primaries_dict["000"] = spectra
paper = cmy_primaries_dict["000"]

In [8]:
names = ["mesh", "lattice"]
# render all objects
viz.renderObjectsPS(mesh_alpha=0.8, lattice_alpha=1)

# transform to correct dir
mat = np.eye(4)
mat[:3, :3] = viz.HMatrix[::-1]
viz.ps.get_surface_mesh("mesh").set_transform(mat)
viz.ps.get_surface_mesh("lattice").set_transform(mat)

In [28]:
# Ideal Printer Gamut
refs, discrete_points, rgbs, lines = maxbasis.getDiscreteRepresentation()
cvpy_idxs = [-2, -3, -4]
collection = list(np.array(refs)[cvpy_idxs])
paper = Spectra(np.stack([wavelengths, np.ones_like(cmy_primaries_dict["000"].data)], axis=1))
gamut = InkGamut(collection, paper, illuminant=d65)
point_cloud, perc = gamut.get_point_cloud(trichromat, stepsize=0.5)
names += viz.renderPrinterGamutConvexHull("cmy", point_cloud, mat=mat)

Generating point cloud: 1it [00:00, 3819.95it/s]


In [47]:
names

['mesh',
 'lattice',
 'cmy_printer_mesh',
 'cmy_printer_mesh',
 'cmy_printer_mesh',
 'cmy_printer_mesh']

In [58]:
long_frames = 180
r = 4
theta = 65
final_phi = 360
offset = 0
filename = "./outputs/CMY_Gamut.mp4"
origin = np.array([0, 0, np.sqrt(3)/2])

types =  ['surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh', 'point_cloud', 'surface_mesh']
map_name_type = {name:typ for name, typ in zip(names, types)}
anim = Animation(viz, map_name_type)
anim.ResetTransparencies()
viz.ps.get_surface_mesh("mesh").set_transparency(0.6)
viz.ps.get_surface_mesh("cmy_printer_mesh").set_transparency(0.2)
viz.ps.get_surface_mesh("cmy_printer_mesh").set_enabled(False)
viz.ps.get_surface_mesh("lattice").set_transparency(0.2)
viz.ps.get_surface_mesh("lattice").set_enabled(False)

In [59]:
fd = viz.openVideo(filename)
rotAroundZ, phi = anim.RotateAroundZ(long_frames//2, r, theta, [0, final_phi//2], lookAt=origin)
offset = viz.renderVideo(rotAroundZ, fd, long_frames//2, offset)

rotAroundZ, phi = anim.RotateAroundZ(long_frames//2, r, theta, [phi, final_phi], lookAt=origin)
fadeInPrinter = anim.FadeIn("cmy_printer_mesh", long_frames//2, [0.2, 1])
fadeInLattice= anim.FadeIn("lattice", long_frames//2, [0.2, 1])
concatAll = anim.concatFns([rotAroundZ, fadeInPrinter, fadeInLattice])

viz.ps.get_surface_mesh("lattice").set_enabled(True)
viz.ps.get_surface_mesh("cmy_printer_mesh").set_enabled(True)
offset = viz.renderVideo(concatAll, fd, long_frames//2, offset)

rotAroundZ, phi = anim.RotateAroundZ(long_frames//2, r, theta, [phi, final_phi//2], lookAt=origin)
offset = viz.renderVideo(rotAroundZ, fd, long_frames//2, offset)

viz.closeVideo(fd)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 